<a href="https://colab.research.google.com/github/codingniket/Python-Training/blob/main/18-12-2025/Excercise_2_18_12_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATASET 1 — USER REGISTRATION
(CORRUPTED SCHEMA)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StructType").getOrCreate()

from pyspark.sql.types import (StructType, StructField, StringType,LongType,IntegerType,ArrayType,MapType)

In [ ]:
raw_users = [
("U001","Amit","28","Hyderabad","['AI','ML','Cloud']"),
("U002","Neha","Thirty","Delhi","AI,Testing"),
("U003","Ravi",None,"Bangalore",["Data","Spark"]),
("U004","Pooja","29","Mumbai",None),
("U005","", "31","Chennai","['DevOps']")
]

In [ ]:
user_schema = StructType([
    StructField("user_id", StringType(), nullable=False),
    StructField("name", StringType(), nullable=True),
    StructField("age", StringType(), nullable=True),
    StructField("city", StringType(), nullable=True),
    StructField("skills", StringType(), nullable=True)
])


skills_schema = StructType([
    StructField("domain", StringType(), nullable=False),
])

print("Defined user and skills schema.")

Defined user and skills schema.


In [ ]:
df_data = spark.createDataFrame(raw_users, user_schema)
df_data.show()
df_data.printSchema()

+-------+-----+------+---------+-------------------+
|user_id| name|   age|     city|             skills|
+-------+-----+------+---------+-------------------+
|   U001| Amit|    28|Hyderabad|['AI','ML','Cloud']|
|   U002| Neha|Thirty|    Delhi|         AI,Testing|
|   U003| Ravi|  NULL|Bangalore|      [Data, Spark]|
|   U004|Pooja|    29|   Mumbai|               NULL|
|   U005|     |    31|  Chennai|         ['DevOps']|
+-------+-----+------+---------+-------------------+

root
 |-- user_id: string (nullable = false)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- city: string (nullable = true)
 |-- skills: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col, when


clean_data = df_data.withColumn(
    "name",
    when(col("name") == "", None).otherwise(col("name"))
)

clean_data = clean_data.withColumn("age", when(col("age") == "", None)
    .when(col("age").rlike(r"^\d+$"),
          col("age").cast(IntegerType()))
    .otherwise(None))

clean_data.show()
clean_data.printSchema()

+-------+-----+----+---------+-------------------+
|user_id| name| age|     city|             skills|
+-------+-----+----+---------+-------------------+
|   U001| Amit|  28|Hyderabad|['AI','ML','Cloud']|
|   U002| Neha|NULL|    Delhi|         AI,Testing|
|   U003| Ravi|NULL|Bangalore|      [Data, Spark]|
|   U004|Pooja|  29|   Mumbai|               NULL|
|   U005| NULL|  31|  Chennai|         ['DevOps']|
+-------+-----+----+---------+-------------------+

root
 |-- user_id: string (nullable = false)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- skills: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col, when, regexp_replace, split, trim, array_compact, transform
from pyspark.sql.types import ArrayType, StringType

clean_data = clean_data.withColumn(
    "skills",
    (when(
        col("skills").isNull(),
        None
    ).otherwise(
        array_compact(
            transform(
                split(
                    regexp_replace(col("skills"), "[\[\]']", ""),
                    ","
                ),
                lambda x: trim(x)
            )
        )
    )).cast(ArrayType(StringType()))
)

clean_data.show(truncate=False)
clean_data.printSchema()

<>:13: SyntaxWarning: invalid escape sequence '\['
<>:13: SyntaxWarning: invalid escape sequence '\['
/tmp/ipython-input-3482429054.py:13: SyntaxWarning: invalid escape sequence '\['
  regexp_replace(col("skills"), "[\[\]']", ""),


+-------+-----+----+---------+---------------+
|user_id|name |age |city     |skills         |
+-------+-----+----+---------+---------------+
|U001   |Amit |28  |Hyderabad|[AI, ML, Cloud]|
|U002   |Neha |NULL|Delhi    |[AI, Testing]  |
|U003   |Ravi |NULL|Bangalore|[Data, Spark]  |
|U004   |Pooja|29  |Mumbai   |NULL           |
|U005   |NULL |31  |Chennai  |[DevOps]       |
+-------+-----+----+---------+---------------+

root
 |-- user_id: string (nullable = false)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
users_df = clean_data

DATASET 2 — COURSE CATALOG (NESTED
STRUCT)

In [ ]:
data2 = [
 ("C001","PySpark Mastery",("Data Engineering","Advanced"),"₹9999"),
 ("C002","AI for Testers",{"domain":"QA","level":"Beginner"},"8999"),
 ("C003","ML Foundations",("AI","Intermediate"),None),
 ("C004","Data Engineering Bootcamp","Data|Advanced","₹14999")
]

1. Create nested StructType for course metadata
2. Normalize domain and level
3. Convert price to IntegerType
4. Handle missing prices
5. Produce courses_df

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StructType").getOrCreate()

from pyspark.sql.types import (StructType, StructField, StringType,LongType,IntegerType,ArrayType,MapType)

In [ ]:
course_schema = StructType([
    StructField("course_id", StringType(), nullable=False),
    StructField("course", StringType(), nullable=True),
    StructField("skills", StringType(), nullable=True),
     StructField("amount", StringType(), nullable=True),
])


In [ ]:
def normalize_skills(skill):
    if isinstance(skill, (list, tuple)):
        return "|".join(skill)
    elif isinstance(skill, dict):
        return "|".join(skill.values())
    else:
        return skill
data2_cleaned = [
    (u, c, normalize_skills(s), a)
    for u, c, s, a in data2
]
df_clean = spark.createDataFrame(data2_cleaned, course_schema)
df_clean.show(truncate=False)

+---------+-------------------------+-------------------------+------+
|course_id|course                   |skills                   |amount|
+---------+-------------------------+-------------------------+------+
|C001     |PySpark Mastery          |Data Engineering|Advanced|₹9999 |
|C002     |AI for Testers           |QA|Beginner              |8999  |
|C003     |ML Foundations           |AI|Intermediate          |NULL  |
|C004     |Data Engineering Bootcamp|Data|Advanced            |₹14999|
+---------+-------------------------+-------------------------+------+



In [ ]:
from pyspark.sql.functions import col, when, substring, split, trim,regexp_replace
clean_data = df_clean.withColumn(
    "amount",
    when(col("amount").isNull() , 0 ).otherwise(regexp_replace(col("amount"), "₹", ""))
    .cast('int')
)


clean_data.show(truncate=False)
clean_data.printSchema()

+---------+-------------------------+-------------------------+------+
|course_id|course                   |skills                   |amount|
+---------+-------------------------+-------------------------+------+
|C001     |PySpark Mastery          |Data Engineering|Advanced|9999  |
|C002     |AI for Testers           |QA|Beginner              |8999  |
|C003     |ML Foundations           |AI|Intermediate          |0     |
|C004     |Data Engineering Bootcamp|Data|Advanced            |14999 |
+---------+-------------------------+-------------------------+------+

root
 |-- course_id: string (nullable = false)
 |-- course: string (nullable = true)
 |-- skills: string (nullable = true)
 |-- amount: integer (nullable = true)



In [ ]:
df_array = clean_data.withColumn(
    "skills",
    split(col("skills"), "\\|")
)

clean_data.show(truncate=False)
clean_data.printSchema()

+---------+-------------------------+-------------------------+------+
|course_id|course                   |skills                   |amount|
+---------+-------------------------+-------------------------+------+
|C001     |PySpark Mastery          |Data Engineering|Advanced|9999  |
|C002     |AI for Testers           |QA|Beginner              |8999  |
|C003     |ML Foundations           |AI|Intermediate          |0     |
|C004     |Data Engineering Bootcamp|Data|Advanced            |14999 |
+---------+-------------------------+-------------------------+------+

root
 |-- course_id: string (nullable = false)
 |-- course: string (nullable = true)
 |-- skills: string (nullable = true)
 |-- amount: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import col, create_map
from pyspark.sql.types import MapType, StringType

df_map = df_array.withColumn(
    "skills",
    create_map(col("skills").getItem(0), col("skills").getItem(1)).cast(MapType(StringType(), StringType()))
)

df_map.show(truncate=False)
df_map.printSchema()

+---------+-------------------------+------------------------------+------+
|course_id|course                   |skills                        |amount|
+---------+-------------------------+------------------------------+------+
|C001     |PySpark Mastery          |{Data Engineering -> Advanced}|9999  |
|C002     |AI for Testers           |{QA -> Beginner}              |8999  |
|C003     |ML Foundations           |{AI -> Intermediate}          |0     |
|C004     |Data Engineering Bootcamp|{Data -> Advanced}            |14999 |
+---------+-------------------------+------------------------------+------+

root
 |-- course_id: string (nullable = false)
 |-- course: string (nullable = true)
 |-- skills: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- amount: integer (nullable = true)



In [ ]:
courses_df = df_map

DATASET 3 — USER COURSE ENROLLMENTS
(JOIN + BROADCAST)

In [ ]:
data3 = [
 ("U001","C001","2024-01-05"),
 ("U002","C002","05/01/2024"),
 ("U003","C001","2024/06/01"),
 ("U004","C003","invalid_date"),
 ("U001","C004","2024-01-10")
]


In [ ]:
enroll_schema = StructType([
    StructField("user_id", StringType(), False),
    StructField("course_id", StringType(), False),
    StructField("enroll_date_raw", StringType(), True)
])

enroll_df = spark.createDataFrame(data3, enroll_schema)

enroll_df.show(truncate=False)
enroll_df.printSchema()

+-------+---------+---------------+
|user_id|course_id|enroll_date_raw|
+-------+---------+---------------+
|U001   |C001     |2024-01-05     |
|U002   |C002     |05/01/2024     |
|U003   |C001     |2024/06/01     |
|U004   |C003     |invalid_date   |
|U001   |C004     |2024-01-10     |
+-------+---------+---------------+

root
 |-- user_id: string (nullable = false)
 |-- course_id: string (nullable = false)
 |-- enroll_date_raw: string (nullable = true)



In [ ]:
from pyspark.sql.functions import coalesce, col, try_to_timestamp, lit
from pyspark.sql.types import DateType

enroll_normalized = enroll_df.withColumn(
    "enroll_date",
    coalesce(
        try_to_timestamp(col("enroll_date_raw"), lit("yyyy-MM-dd")).cast(DateType()),
        try_to_timestamp(col("enroll_date_raw"), lit("dd/MM/yyyy")).cast(DateType()),
        try_to_timestamp(col("enroll_date_raw"), lit("yyyy/MM/dd")).cast(DateType())
    )
).drop("enroll_date_raw")

enroll_normalized.show(truncate=False)
enroll_normalized.printSchema()

+-------+---------+-----------+
|user_id|course_id|enroll_date|
+-------+---------+-----------+
|U001   |C001     |2024-01-05 |
|U002   |C002     |2024-01-05 |
|U003   |C001     |2024-06-01 |
|U004   |C003     |NULL       |
|U001   |C004     |2024-01-10 |
+-------+---------+-----------+

root
 |-- user_id: string (nullable = false)
 |-- course_id: string (nullable = false)
 |-- enroll_date: date (nullable = true)



In [ ]:
valid_enrollments = enroll_normalized.filter(
    col("enroll_date").isNotNull()
)


enroll_users_df = valid_enrollments.join(
    users_df,
    on="user_id",
    how="left"
)

final_df = enroll_users_df.join(
    courses_df,
    on="course_id",
    how="left"
)

final_df.show(truncate=False)

# users_df.show(truncate=False)
# courses_df.show(truncate=False)


+---------+-------+-----------+------+------+------+-------------------------+------------------------------+------+
|course_id|user_id|enroll_date|course|skills|amount|course                   |skills                        |amount|
+---------+-------+-----------+------+------+------+-------------------------+------------------------------+------+
|C002     |U002   |2024-01-05 |NULL  |NULL  |NULL  |AI for Testers           |{QA -> Beginner}              |8999  |
|C001     |U001   |2024-01-05 |NULL  |NULL  |NULL  |PySpark Mastery          |{Data Engineering -> Advanced}|9999  |
|C001     |U003   |2024-06-01 |NULL  |NULL  |NULL  |PySpark Mastery          |{Data Engineering -> Advanced}|9999  |
|C004     |U001   |2024-01-10 |NULL  |NULL  |NULL  |Data Engineering Bootcamp|{Data -> Advanced}            |14999 |
+---------+-------+-----------+------+------+------+-------------------------+------------------------------+------+



DATASET 4 — USER ACTIVITY LOGS (ARRAY +
MAP)

Exercises
1. Normalize actions into ArrayType
2. Normalize metadata into MapType
3. Handle missing actions safely
4. Explode actions and count frequency
5. Produce activity_df


In [74]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, when, explode, from_json, create_map, lit
from pyspark.sql.types import ArrayType, StringType, MapType
import json

spark = SparkSession.builder.getOrCreate()

# Raw data
raw_activity = [
    ("U001","login,watch,logout","{'device':'mobile','ip':'1.1.1.1'}",120),
    ("U002",["login","watch"],"device=laptop;ip=2.2.2.2",90),
    ("U003","login|logout",None,30),
    ("U004",None,"{'device':'tablet'}",60)
]


course_schema = StructType([
    StructField("user_id", StringType(), nullable=False),
    StructField("actions", StringType(), nullable=True),
    StructField("metadata", StringType(), nullable=True),
     StructField("duration", IntegerType(), nullable=True),
])

df = spark.createDataFrame(raw_activity, course_schema)

df.show(truncate=False)
df.printSchema()

+-------+------------------+----------------------------------+--------+
|user_id|actions           |metadata                          |duration|
+-------+------------------+----------------------------------+--------+
|U001   |login,watch,logout|{'device':'mobile','ip':'1.1.1.1'}|120     |
|U002   |[login, watch]    |device=laptop;ip=2.2.2.2          |90      |
|U003   |login|logout      |NULL                              |30      |
|U004   |NULL              |{'device':'tablet'}               |60      |
+-------+------------------+----------------------------------+--------+

root
 |-- user_id: string (nullable = false)
 |-- actions: string (nullable = true)
 |-- metadata: string (nullable = true)
 |-- duration: integer (nullable = true)



In [77]:
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType,
    ArrayType, MapType
)
from pyspark.sql.functions import (
    col, split, regexp_replace, trim, when, coalesce,
    from_json, explode, count, lit, array, size,
    expr, udf
)
df_raw = df
df_with_actions = df_raw.withColumn(
    "actions_normalized",
    when(col("actions").isNull(), array())  # Empty array for null
    .when(col("actions").contains("['"),
          # Handle Python list format: ['login', 'watch']
          split(
              regexp_replace(
                  regexp_replace(col("actions"), "\\[|\\]|'", ""),
                  " ", ""
              ),
              ","
          )
    )
    .when(col("actions").contains("|"),
          # Handle pipe delimiter
          split(col("actions"), "\\|")
    )
    .otherwise(
          # Handle comma delimiter (default)
          split(col("actions"), ",")
    )
).withColumn(
    # Trim whitespace from each action
    "actions_array",
    expr("transform(actions_normalized, x -> trim(x))")
).withColumn(
    # Filter out empty strings
    "actions_array",
    expr("filter(actions_array, x -> x != '')")
)

In [79]:
df_with_actions.show(truncate=False)

+-------+------------------+----------------------------------+--------+----------------------+----------------------+
|user_id|actions           |metadata                          |duration|actions_normalized    |actions_array         |
+-------+------------------+----------------------------------+--------+----------------------+----------------------+
|U001   |login,watch,logout|{'device':'mobile','ip':'1.1.1.1'}|120     |[login, watch, logout]|[login, watch, logout]|
|U002   |[login, watch]    |device=laptop;ip=2.2.2.2          |90      |[[login,  watch]]     |[[login, watch]]      |
|U003   |login|logout      |NULL                              |30      |[login, logout]       |[login, logout]       |
|U004   |NULL              |{'device':'tablet'}               |60      |[]                    |[]                    |
+-------+------------------+----------------------------------+--------+----------------------+----------------------+



In [82]:
def parse_metadata(metadata_str) -> dict[str, str]:
    if metadata_str is None:
        return {}

    metadata_str = metadata_str.strip()
    result = {}

    try:
        # Try parsing as JSON-like string (with single quotes)
        if metadata_str.startswith("{") and metadata_str.endswith("}"):
            # Replace single quotes with double quotes for valid JSON
            json_str = metadata_str.replace("'", '"')
            result = json.loads(json_str)
        # Parse key=value;key=value format
        elif "=" in metadata_str:
            pairs = metadata_str.split(";")
            for pair in pairs:
                if "=" in pair:
                    key, value = pair.split("=", 1)
                    result[key.strip()] = value.strip()
    except Exception as e:
        # Return empty dict if parsing fails
        pass

    return result


In [83]:
parse_metadata_udf = udf(parse_metadata, MapType(StringType(), StringType()))

df_with_metadata = df_with_actions.withColumn(
    "metadata_map",
    parse_metadata_udf(col("metadata"))
)

In [84]:
activity_df = df_with_metadata.select(
    col("user_id"),
    col("actions_array").alias("actions"),
    col("metadata_map").alias("metadata"),
    col("duration")
).withColumn(
    "action_count",
    size(col("actions"))
).withColumn(
    "has_actions",
    col("action_count") > 0
)


In [85]:
activity_df.show(truncate=False)
activity_df.printSchema()

+-------+----------------------+---------------------------------+--------+------------+-----------+
|user_id|actions               |metadata                         |duration|action_count|has_actions|
+-------+----------------------+---------------------------------+--------+------------+-----------+
|U001   |[login, watch, logout]|{device -> mobile, ip -> 1.1.1.1}|120     |3           |true       |
|U002   |[[login, watch]]      |{device -> laptop, ip -> 2.2.2.2}|90      |2           |true       |
|U003   |[login, logout]       |{}                               |30      |2           |true       |
|U004   |[]                    |{device -> tablet}               |60      |0           |false      |
+-------+----------------------+---------------------------------+--------+------------+-----------+

root
 |-- user_id: string (nullable = false)
 |-- actions: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- metadata: map (nullable = true)
 |    |-- key: string


In [86]:
df_exploded = activity_df.filter(col("has_actions") == True).select(
    col("user_id"),
    explode(col("actions")).alias("action"),
    col("metadata"),
    col("duration")
)

In [88]:
action_frequency = df_exploded.groupBy("action").agg(
    count("*").alias("frequency")
).orderBy(col("frequency").desc())

print("\n" + "=" * 80)
print("ACTION FREQUENCY")
print("=" * 80)
action_frequency.show(truncate=False)

# User-Action frequency (actions per user)
user_action_frequency = df_exploded.groupBy("user_id", "action").agg(
    count("*").alias("frequency")
).orderBy("user_id", col("frequency").desc())

print("\n" + "=" * 80)
print("USER-ACTION FREQUENCY")
print("=" * 80)
user_action_frequency.show(truncate=False)


ACTION FREQUENCY
+------+---------+
|action|frequency|
+------+---------+
|logout|2        |
|login |2        |
|watch |1        |
|[login|1        |
|watch]|1        |
+------+---------+


USER-ACTION FREQUENCY
+-------+------+---------+
|user_id|action|frequency|
+-------+------+---------+
|U001   |login |1        |
|U001   |watch |1        |
|U001   |logout|1        |
|U002   |watch]|1        |
|U002   |[login|1        |
|U003   |logout|1        |
|U003   |login |1        |
+-------+------+---------+



DATASET 5 — PAYMENTS (WINDOW +
AGGREGATES)

Exercises
1. Convert dates properly
2. Compute total spend per user (GroupBy)
3. Compute running spend per user (Window)
4. Rank users by total spend
5. Compare GroupBy vs Window outputs

In [89]:
raw_payments = [
 ("U001","2024-01-05",9999),
 ("U001","2024-01-10",14999),
 ("U002","2024-01-06",8999),
 ("U003","2024-01-07",0),
 ("U004","2024-01-08",7999),
 ("U001","2024-01-15",1999)
]


In [90]:
schema = StructType([
    StructField("user_id", StringType(), False),
    StructField("payment_date", StringType(), False),
    StructField("amount", IntegerType(), False)
])

df = spark.createDataFrame(raw_payments, schema)

In [93]:
df_processed = df.withColumn(
    "date_normalized",
    coalesce(
        try_to_timestamp(col("payment_date"), lit("yyyy-MM-dd")).cast(DateType()),
        try_to_timestamp(col("payment_date"), lit("dd/MM/yyyy")).cast(DateType()),
        try_to_timestamp(col("payment_date"), lit("yyyy/MM/dd")).cast(DateType())
    ))

df_processed.show(truncate=False)
df_processed.printSchema()

+-------+------------+------+---------------+
|user_id|payment_date|amount|date_normalized|
+-------+------------+------+---------------+
|U001   |2024-01-05  |9999  |2024-01-05     |
|U001   |2024-01-10  |14999 |2024-01-10     |
|U002   |2024-01-06  |8999  |2024-01-06     |
|U003   |2024-01-07  |0     |2024-01-07     |
|U004   |2024-01-08  |7999  |2024-01-08     |
|U001   |2024-01-15  |1999  |2024-01-15     |
+-------+------------+------+---------------+

root
 |-- user_id: string (nullable = false)
 |-- payment_date: string (nullable = false)
 |-- amount: integer (nullable = false)
 |-- date_normalized: date (nullable = true)



In [96]:
from pyspark.sql.functions import (
    col, sum, count, avg, max, min, coalesce, lit,
    row_number, rank, dense_rank, percent_rank
)
total_spend_per_user = df_processed.groupBy("user_id").agg(
    sum("amount").alias("total_spend"),
    count("amount").alias("transaction_count"),
    avg("amount").alias("avg_spend"),
    max("amount").alias("max_spend"),
    min("amount").alias("min_spend")
).orderBy(col("total_spend").desc())

total_spend_per_user.show(truncate=False)

+-------+-----------+-----------------+---------+---------+---------+
|user_id|total_spend|transaction_count|avg_spend|max_spend|min_spend|
+-------+-----------+-----------------+---------+---------+---------+
|U001   |26997      |3                |8999.0   |14999    |1999     |
|U002   |8999       |1                |8999.0   |8999     |8999     |
|U004   |7999       |1                |7999.0   |7999     |7999     |
|U003   |0          |1                |0.0      |0        |0        |
+-------+-----------+-----------------+---------+---------+---------+



In [99]:
from pyspark.sql.window import Window
window_spec = Window.partitionBy("user_id").orderBy("date_normalized")

# Calculate running totals and other window analytics
df_with_running_totals = df_processed.withColumn(
    "running_total",
    sum("amount").over(window_spec)
).withColumn(
    "transaction_number",
    row_number().over(window_spec)
).withColumn(
    "cumulative_avg",
    avg("amount").over(window_spec)
)

df_with_running_totals.show(truncate=False)

+-------+------------+------+---------------+-------------+------------------+--------------+
|user_id|payment_date|amount|date_normalized|running_total|transaction_number|cumulative_avg|
+-------+------------+------+---------------+-------------+------------------+--------------+
|U001   |2024-01-05  |9999  |2024-01-05     |9999         |1                 |9999.0        |
|U001   |2024-01-10  |14999 |2024-01-10     |24998        |2                 |12499.0       |
|U001   |2024-01-15  |1999  |2024-01-15     |26997        |3                 |8999.0        |
|U002   |2024-01-06  |8999  |2024-01-06     |8999         |1                 |8999.0        |
|U003   |2024-01-07  |0     |2024-01-07     |0            |1                 |0.0           |
|U004   |2024-01-08  |7999  |2024-01-08     |7999         |1                 |7999.0        |
+-------+------------+------+---------------+-------------+------------------+--------------+



In [100]:
from pyspark.sql.functions.builtin import user
window_rank_spec = Window.orderBy(col("total_spend").desc())

user_rankings = total_spend_per_user.withColumn(
    "rank",
    rank().over(window_rank_spec)
).withColumn(
    "dense_rank",
    dense_rank().over(window_rank_spec)
).withColumn(
    "row_number",
    row_number().over(window_rank_spec)
).withColumn(
    "percent_rank",
    percent_rank().over(window_rank_spec)
)

user_rankings.show(truncate=False)

+-------+-----------+-----------------+---------+---------+---------+----+----------+----------+------------------+
|user_id|total_spend|transaction_count|avg_spend|max_spend|min_spend|rank|dense_rank|row_number|percent_rank      |
+-------+-----------+-----------------+---------+---------+---------+----+----------+----------+------------------+
|U001   |26997      |3                |8999.0   |14999    |1999     |1   |1         |1         |0.0               |
|U002   |8999       |1                |8999.0   |8999     |8999     |2   |2         |2         |0.3333333333333333|
|U004   |7999       |1                |7999.0   |7999     |7999     |3   |3         |3         |0.6666666666666666|
|U003   |0          |1                |0.0      |0        |0        |4   |4         |4         |1.0               |
+-------+-----------+-----------------+---------+---------+---------+----+----------+----------+------------------+

